### Install Transformers

In [ ]:
!pip3 install --quiet transformers

In [ ]:
import tensorflow as tf

### Data Processing

Load data

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
data = pd.read_csv("/content/drive/My Drive/NLP/Project/NLPMiniProject_data.csv")
print(data.head())

In [ ]:
df = data[['ID','comment','parent_comment','down','score','top','topic','label']]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df['comment']+df['parent_comment'],
    df['label'],
    test_size=0.2, 
    random_state=42
)

In [ ]:
X_train.shape, X_test.shape

Tokenize data using Bert Tokenizer

In [ ]:
!pip install sentencepiece

In [ ]:
from transformers import *

In [ ]:
#Get BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
#tokenizer.vocab.items()

In [ ]:
sentences = (df['comment']+df['parent_comment']).values

In [ ]:
labels = df.label.values

In [ ]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

In [ ]:
sentences[0]

In [ ]:
#Check tokenized text
print(tokenized_texts[0])

In [ ]:
#We will use only first 200 tokens to do classification (this value can be changed)
max_length = 200
tokenized_texts = [sent[:max_length] for sent in tokenized_texts]

In [ ]:
for i in range(len(tokenized_texts)):
    sent = tokenized_texts[i]
    sent = ['[CLS]'] + sent + ['[SEP]']
    tokenized_texts[i] = sent

In [ ]:
print(tokenized_texts[0])

In [ ]:
#Convert tokens into IDs
input_ids = [tokenizer.convert_tokens_to_ids(sent) for sent in tokenized_texts]

In [ ]:
print(input_ids[0])

In [ ]:
tokenizer.vocab.items()

In [ ]:
#Pad our tokens which might be less than max_length size
input_ids = tf.keras.preprocessing.sequence.pad_sequences(input_ids, maxlen=max_length+2, truncating='post', padding='post')

In [ ]:
input_ids.shape

Split data between training and test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#80% data will be used for training while 20% will be used for test
trainX, testX, trainY, testY = train_test_split(input_ids, labels, test_size=0.2, random_state=12345)

Create Attention masks : Attention masks are useful to ignore padding tokens. Mask value will be set to 0 for padding tokens and 1 for actual tokens. We will create mask both for training and test data

In [ ]:
# Create attention masks for training
train_attn_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in trainX:
  seq_mask = [float(i>0) for i in seq]
  train_attn_masks.append(seq_mask)

In [ ]:
# Create attention masks for Test
test_attn_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in testX:
  seq_mask = [float(i>0) for i in seq]
  test_attn_masks.append(seq_mask)

In [ ]:
print(train_attn_masks[100])

At this point, we have the data ready

### Build Model

In [ ]:
#Load Pre-trained Bert Model with a Binary Classification layer at the top.
#Huggingface library provides TFBertForSequenceClassification for the same
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

In [ ]:
model.summary()

In [ ]:
# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule 
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)

In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

### Train Model

In [ ]:
import numpy as np

In [ ]:
train_x_data = {'input_ids': np.array(trainX), 'attention_mask': np.array(train_attn_masks)}
test_x_data = {'input_ids': np.array(testX), 'attention_mask': np.array(test_attn_masks)}

In [ ]:
model.fit(train_x_data, trainY, 
          validation_data=(test_x_data, testY), 
          batch_size=16, 
          epochs=2)